In [8]:
import pyforms
from   pyforms          import BaseWidget
from   pyforms.controls import ControlText
from   pyforms.controls import ControlButton
from tkinter import *
from tkinter import filedialog, messagebox, PhotoImage
from tkinter.filedialog import askopenfilename,askdirectory
import cv2
import numpy as np
import tensorflow as tf
import PIL
from PIL import Image, ImageTk
import os

from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import numpy as np
import cv2
import utils
import argparse
import webcolors
import numpy as np
import cv2

In [45]:
class window:
    def __init__(self):
        self.root = Tk()
        #self.image = str()
        self.imagem_cor = str()
        self.crop_img = str()
        self.path_ = str()
        self.widgets()
        self.root.mainloop()
        
    
    def closest_colour(self,requested_colour):
        min_colours = {}
        for key, name in webcolors.CSS3_HEX_TO_NAMES.items():
            r_c, g_c, b_c = webcolors.hex_to_rgb(key)
            rd = (r_c - requested_colour[0]) ** 2
            gd = (g_c - requested_colour[1]) ** 2
            bd = (b_c - requested_colour[2]) ** 2
            min_colours[(rd + gd + bd)] = name
        return min_colours[min(min_colours.keys())]

    def get_colour_name(self,requested_colour):
        try:
            closest_name = actual_name = webcolors.rgb_to_name(requested_colour)
        except ValueError:
            closest_name = self.closest_colour(requested_colour)
            actual_name = None
        return actual_name, closest_name

    # load the image and convert it from BGR to RGB so that
    # we can dispaly it with matplotlib
    def crop_image(self):
        image = cv2.imread(self.path_)
        center = image.shape
        y=500
        x=250
        h=500
        w=1500
        #print(h,w)
        crop_img=image[y:y+h,x:x+w]
        self.crop_img = cv2.cvtColor(crop_img, cv2.COLOR_BGR2RGB)
        # show our image
        plt.figure()
        plt.axis("off")
        plt.imshow(self.crop_img)
        #return self.image

    # import the necessary packages
    def centroid_histogram(self,clt):
        # grab the number of different clusters and create a histogram
        # based on the number of pixels assigned to each cluster
        numLabels = np.arange(0, len(np.unique(clt.labels_)) + 1)
        (hist, _) = np.histogram(clt.labels_, bins = numLabels)
        # normalize the histogram, such that it sums to one
        hist = hist.astype("float")
        hist /= hist.sum()
        # return the histogram
        return hist

    def plot_colors(self,hist, centroids):
        # initialize the bar chart representing the relative frequency
        # of each of the colors
        bar = np.zeros((50, 300, 3), dtype = "uint8")
        startX = 0
        # loop over the percentage of each cluster and the color of
        # each cluster
        max = 0
        max_colour = 0.000
        for (percent, color) in zip(hist, centroids):
            # plot the relative percentage of each cluster
            if percent > max:
                max = percent
                max_colour = self.get_colour_name(color.astype("uint8").tolist())
            #print(get_colour_name(color.astype("uint8").tolist()),percent)
            endX = startX + (percent * 300)
            cv2.rectangle(bar, (int(startX), 0), (int(endX), 50),
                color.astype("uint8").tolist(), -1)
            startX = endX
        # return the bar chart
        # show our color bart
        plt.figure()
        plt.axis("off")
        plt.imshow(bar)
        plt.show()
        print(max_colour[1])
        return bar, max_colour
    
    
    def CNN(self):
        # Previsão através de uma imagem
        #definir tamanho das imagens
        nlinhas = 256
        ncolunas = 256
        canais = 3  #dimensões das cores (1 para preto e branco e 3 para colorido)

        # Recreate the exact same model, including its weights and the optimizer
        model = tf.keras.models.load_model("C:/Users/jgoncalves/Desktop/modelo_cnn_modelos/marca_modelo_sgd3.h5")
        self.imagem_cor = cv2.resize(cv2.imread(self.path_, cv2.IMREAD_COLOR), (nlinhas,ncolunas), interpolation=cv2.INTER_CUBIC)
        #imagem_peb = cv2.cvtColor(imagem_cor, cv2.COLOR_BGR2GRAY)
        #tf.expand_dims(
        imagem = tf.expand_dims(np.array(self.imagem_cor).astype("float32") / 255, 0)   
            #,3)
        pred = model.predict(imagem)
        pred = np.argmax(pred)
        marcas = ['Citan','CLA','ClasseA','ClasseB','ClasseC(205)','ClasseC(206)','ClasseE','ClasseS','CLS','EQA','EQB','EQC','EQE','EQS','ForFour','ForTwo','GLA','GLA(156)','GLB','GLC','GLE','SL','Vito']
        #plt.figure(figsize=(5,2.5))
        #plt.imshow(imagem_cor)

        self.crop_image()

        # reshape the image to be a list of pixels
        self.crop_img = self.crop_img.reshape((self.crop_img.shape[0] * self.crop_img.shape[1], 3))
        # cluster the pixel intensities
        clt = KMeans(n_clusters =3)
        clt.fit(self.crop_img)

        # build a histogram of clusters and then create a figure
        # representing the number of pixels labeled to each color
        hist = self.centroid_histogram(clt)
        bar, max_colour = self.plot_colors(hist, clt.cluster_centers_)

        # return the bar chart
        # show our color bart
        plt.figure()
        plt.axis("off")
        plt.imshow(bar)
        plt.show()

        return messagebox.askquestion('Leonardo Banana',"Previu-se ser " + marcas[pred] + ".\n" "Cor: {}".format(max_colour[1]) )
    
    def focus1(self,event):
        # set focus on the course_field box
        self.path_ = filedialog.askopenfilenames(title = 'open')
        self.path_ = str(self.path_[0])
        self.path_field.insert(0,self.path_)
        print(self.path_)
        self.show_image()
        #try:
    
    def focus2(self,event):
        self.CNN()
    
    def focus3(self,event):
        self.path_field.delete(0,END)
    
    
    def widgets(self):
        self.setup()
    
    def show_image(self):
        self.imagem_cor = cv2.resize(cv2.imread(self.path_, cv2.IMREAD_COLOR), (256,256), interpolation=cv2.INTER_CUBIC)
        logo =PhotoImage(self.imagem_cor) # displays logo.gif in a label widget
        logoimage = Label(image = logo) # this can't be changed while the program is running 
        logoimage.pack() # position and dimentions

        
    
    def setup(self):
        self.root.configure(background='dark blue')
        self.root.title("Maia Tarot")
        self.root.geometry("500x300")
        
        
        self.path = Label(self.root,text='Image',bg='white')
        self.path.grid(row = 1, column = 0,sticky = N,padx = 20,pady = 20)
        self.path_field = Entry(self.root, textvariable = self.path_) 
        self.path_field.bind("<Button>",self.focus1)
        self.path_field.grid(row=1,column=1,ipadx='100',sticky = N,padx = 20,pady = 20)
        
        self.campo = Canvas(self.root,width = 300, height = 150,bg = 'white')
        self.campo.grid(row = 3,column = 1,sticky = W,pady = 20)
        
        
        self.submit = Button(self.root, text = 'Clica aqui', fg='Black',bg='Grey')
        self.submit.bind("<Button>",self.focus2)
        self.submit.grid(row=2, column=1,sticky = N)
        self.button2 = Button(self.root,text = 'Reset',fg = 'Black',bg='Grey')
        self.button2.bind('<Button>',self.focus3)
        self.button2.grid(row=2, column=0,sticky = N)
        
        
if __name__ == '__main__':
    win = window()
        
    
